In [1]:
import numpy as np
import pandas as pd
import re
import ast
import matplotlib.pyplot as plt
#!pip install wordcloud


Matplotlib is building the font cache; this may take a moment.


In [15]:
path = input('Enter path to master dataset')
path = 'MasterDatasetTemp_Preprocessed_Anonymized_small.csv'
df = pd.read_csv (path) 

In [16]:
df.columns
df.CompanyName_New.isnull().value_counts()
df = df[df['CompanyName_New'].notna()]
df.fillna('', inplace=True)

In [17]:
df_domains = df[['CompanyName_New','ProjectDetailDomain','InternshipProjectDomain','ResearchDomain']]

In [18]:
df_domains['AllDomains'] = df_domains['ProjectDetailDomain']+df_domains['InternshipProjectDomain']+df_domains['ResearchDomain']

C:\Users\rames\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_domains

,CompanyName_New,ProjectDetailDomain,InternshipProjectDomain,ResearchDomain,AllDomains
0,walmart labs,,,,
1,themathcompany,cloud computing##operating system##machine lea...,,,cloud computing##operating system##machine lea...
2,nxp semiconductors,cloud computing##operating system##machine lea...,,,cloud computing##operating system##machine lea...
3,intuit,cloud computing##operating system##machine lea...,,,cloud computing##operating system##machine lea...
4,citrix,cloud computing##machine learning##database ma...,,,cloud computing##machine learning##database ma...
5,novarties healthcare,cloud computing##machine learning##web develop...,,,cloud computing##machine learning##web develop...
6,cisco systems,cloud computing##operating system##web develop...,cloud computing##operating system##machine lea...,operating system##machine learning##deep learn...,cloud computing##operating system##web develop...
7,optum,cloud computing##operating system##web develop...,cloud computing##operating system##machine lea...,operating system##machine learning##deep learn...,cloud computing##operating system##web develop...
8,cisco systems,cloud computing##operating system##database ma...,machine learning##natural language processing#...,machine learning##,cloud computing##operating system##database ma...
9,optum,cloud computing##operating system##machine lea...,image processing##software development##,,cloud computing##operating system##machine lea...


In [20]:
#getting distinct list of all courses, after cleaning
domain = df_domains.AllDomains.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
domain_all = domain.index.values.tolist()
domain_all.remove('')
#print(len(sorted(domain_all)))
#sorted(domain_all)

In [23]:
# function to find occurence of specific domains and creating new column for each unqiue domain. Fill with number of projects in that domain
def keyword_matching_domain(keywords,df):

    df = df.join(pd.DataFrame(0, columns=keywords, index=df.index))
    df['TotalDomainCount'] = 0
    #print(df.columns,keywords)
    for key in keywords:
        #print(key)
        for i in range(0, len(df)):
            if(df.iloc[i]['AllDomains'].find(key)!=-1):
                df.iloc[i, df.columns.get_loc(key)] = df.iloc[i, df.columns.get_loc(key)]+ df.iloc[i]['AllDomains'].count(key)
                df.iloc[i, df.columns.get_loc('TotalDomainCount')] = df.iloc[i, df.columns.get_loc('TotalDomainCount')]+ df.iloc[i]['AllDomains'].count(key)                
    return df

In [24]:
df_doms = keyword_matching_domain(domain_all,df_domains)

In [25]:
# getting total number of domains per company, and total sum of each domain for each company
dfDomainCount = df_doms.groupby(['CompanyName_New'])[domain_all].apply(lambda x : x.astype(int).sum())
dfTotDomCount = df_doms.groupby(['CompanyName_New'])['TotalDomainCount'].apply(lambda x : x.astype(int).sum())

In [28]:
result = pd.concat([dfDomainCount, dfTotDomCount], axis=1)
result1 = result[domain_all].div(result.TotalDomainCount, axis=0)
originalRatingsData1 = result1.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'DomainName', 0:'Rating'})